In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="Michaels_test1")
exp = Experiment(workspace=ws, name="udacity-pipeline")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Michaels_test1
Azure region: northeurope
Subscription id: 336509a3-005c-4650-b1e5-e1c99e57a5e1
Resource group: MS_Test


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
aml_compute_target = "compute-ins1"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 0, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

found existing compute target.
Azure Machine Learning Compute attached


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    #"--kernel": choice('linear', 'rbf', 'poly', 'sigmoid'),
    "--C": uniform(0.25,2),
    "--max_iter": choice(range(50,150))
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#if "training" not in os.listdir():
#    os.mkdir("./training")
if "models" not in os.listdir():
    os.mkdir("./models")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', entry_script='train.py', compute_target=aml_compute)
# 

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                         hyperparameter_sampling=ps, 
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=30,
                                         max_concurrent_runs=4,
                                        policy=policy,
                                        estimator=est
                                        )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: 24870a79-6c39-48f2-936c-7630d4fd2f9a
Web View: https://ml.azure.com/experiments/udacity-pipeline/runs/24870a79-6c39-48f2-936c-7630d4fd2f9a?wsid=/subscriptions/336509a3-005c-4650-b1e5-e1c99e57a5e1/resourcegroups/MS_Test/workspaces/Michaels_test1


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [5]:
import joblib
# Get your best run and save the model from that run.
best_hyper_run = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(best_hyper_run, 'models/best_hyper_run.joblib')
### YOUR CODE HERE ###

TypeError: can't pickle _thread.RLock objects

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
filepath = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=filepath)
### YOUR CODE HERE ###

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
train, test = clean_data(ds)
data = pd.concat([train, test])

ModuleNotFoundError: No module named 'train'

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='accuracy',
    training_data=data,
    label_column_name=y,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = experiment.submit(automl_config)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_model = automl_run.get_output()
joblib.dump(best_ml_run, 'models/best_automl_run.joblib')